In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys

sys.path.append('/content/gdrive/My Drive/Colab Notebooks/ECE_Project')
#sys.path.append('/content/gdrive/My Drive/ECE_Project')

In [0]:
import CNN_Lib
import CNN_BuildingBlock_Lib as BB
import CombinationModel as ComboCNN
import numpy as np 
import torch
from torch.autograd import Variable
import random
import pdb
import copy


In [6]:
dir = '/content/gdrive/My Drive/Colab Notebooks/ECE_Project'
#dir = '/content/gdrive/My Drive/ECE_Project'
X_test = np.load(dir + '/X_test.npy')
y_test = np.load(dir + '/y_test.npy')
person_train_valid = np.load(dir + '/person_train_valid.npy')
X_train_valid = np.load(dir + '/X_train_valid.npy')
y_train_valid = np.load(dir + '/y_train_valid.npy')
person_test = np.load(dir + '/person_test.npy')
print ('Training/Valid data shape: {}' .format(X_train_valid.shape))
print ('Test data shape: {}' .format(X_test.shape))
print ('Training/Valid target shape: {}' .format(y_train_valid.shape))
print ('Test target shape: {}' .format(y_test.shape))
print ('Person train/valid shape: {}' .format(person_train_valid.shape))
print ('Person test shape: {}' .format(person_test.shape))


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [0]:
#transform data into torch-readable data types
#2115 number of trials from 9 people
#four possible classes of outputs
#Xtrain = Variable(torch.from_numpy(X_train_valid)).cuda()
#ytrain = Variable(torch.from_numpy(y_train_valid)).cuda()
#Xtest = Variable(torch.from_numpy(X_test)).cuda()
#ytest = Variable(torch.from_numpy(y_test)).cuda()

Xtrain = torch.from_numpy(X_train_valid)
Ytrain = torch.from_numpy(y_train_valid)
Xtest = torch.from_numpy(X_test)
Ytest = torch.from_numpy(y_test)



In [0]:
def map_to_class(input_labels):
  mask1 = (input_labels == 769)*0
  mask2 = (input_labels == 770)*1
  mask3 = (input_labels == 771)*2
  mask4 = (input_labels == 772)*3

  return (mask1 + mask2 + mask3 + mask4)

In [0]:
#-----CNN tweak here----#
num_filters = [64, 64, 32] #64, 32, 32 up to 70% V, and 64, 64, 32 plateud at ~61-63% after Epoch 1 then increased to 67% | 72 max
batch_size = 30   #smaller batch sizes (less than 50) offer a regularization effect | must be less than 423 bc we are using kfold validation of 5
num_classes = 4
L2 = 0.22 #higher than 0.25 and the validation doesnt reach 70
cnnfilter_stride = [2, 1, 1] #big filter stride makes training to acceptable validation take longer. 
cnnfilter_size = [10, 3, 3] #smaller filter size appears to make higher validation from the beginning. lower than 2 is pretty bad?
cnn_padding = [1, 1, 1]
use_bias = [False, True, False]
use_maxpool = [True, True, True] #not using it makes the model overfit
pool_size = [5, 3, 2]
pool_stride = [2, 1, 1]
use_batchnorm = [True, True, True]
eps = [1e-4, 1e-4, 1e-4]
momentum = [0.3, 0.8, 0.5]
affine = [False, False, False]
dropout = [0.13, 0.21, 0.1, 0.55]

#-----OPTIM tweak here----#
learning_rate = 0.0001 #0 we trained faster with this vs 0.0001 

In [0]:
########## CHOOSE LOSS  ###########
loss = torch.nn.CrossEntropyLoss()

######### CHOOSE OPTIMIZER ########
#optim = torch.optim.Adam(combo_model.parameters(), lr=learning_rate, weight_decay=L2)
#optim = torch.optim.Adadelta(combo_model.parameters()) << does worse than Adam, we overfit so much 

In [67]:
######## TRAIN IT ########
#------ With Early Stopping AND K-Fold Validation -----#
stop_now = False
loss_store = []
validation_store = []
training_store = []
network_store = []
model_store = []

epochs = 1
kfolds = 15
iterations = 1500
iter_ = 0
validate_fold = 0
SEED = 2000

#k fold validation here
np.random.seed(random.randint(1,SEED))
fold_size = int(Trials/kfolds)
idx = np.arange(Trials)
np.random.shuffle(idx)
Xtrain_Shuffled = Xtrain[idx]
Ytrain_Shuffled = Ytrain[idx]

FoldsX = Xtrain_Shuffled.split(fold_size)
FoldsY = Ytrain_Shuffled.split(fold_size)

for epoch in range(epochs):
  if stop_now:
    break

  print(f"...... Training for Epoch {epoch} ......")
  train_correct = 0
  train_total = 0


  for k in range(kfolds): 

    #create training folds by excluding validate fold
    train_folds = list(range(kfolds))
    del train_folds[validate_fold]
    train_folds = np.array(train_folds)

    TrainX = torch.cat([FoldsX[f] for f in train_folds]) 
    TrainY = torch.cat([FoldsY[f] for f in train_folds])
    #create validate fold
    ValidateX = FoldsX[validate_fold]
    ValidateY = FoldsY[validate_fold]

    #initialize the network within the kfold loop
    combo_model = ComboCNN.CombinationModel(InputDim, SeqDim, num_filters, cnnfilter_size, 
                                            cnnfilter_stride, cnn_padding, use_bias, 
                                            num_classes, use_maxpool, pool_size, pool_stride, 
                                            use_batchnorm, eps, momentum, affine, dropout)
    combo_model.to('cuda:0') #--- activate the GPU
    optim = torch.optim.Adam(combo_model.parameters(), lr=learning_rate, weight_decay=L2) 
    

    print(f"______ Training for k-folds {train_folds} ______")
    for i in range(iterations):
      #do the batches
      idx = np.arange(TrainX.size(0))
      np.random.shuffle(idx)
      idx = idx[0:batch_size]

      #process the input data
      xtrain = TrainX[idx].view(batch_size, InputDim, SeqDim)
      xtrain = xtrain.to('cuda:0').requires_grad_()
      ytrain = TrainY[idx]

      #put the model in training mode
      combo_model.train(True)

      #forward pass
      optim.zero_grad()
      outFC = combo_model.forward(xtrain.float())
  
      #map to classes
      classes = map_to_class(ytrain.long())
      classes = classes.to('cuda:0')

      #backward pass (gradient calculation)
      probs = loss(outFC, classes)
      probs.backward()

      #update weights
      optim.step()

      #update iter counter
      iter_+=1

      #calculate training accuracy
      values, train_pred = torch.max(outFC, 1)
      train_correct += (1*(train_pred == classes)).sum()
      train_total += float(classes.size(0))

    print(f"______ Validating for k-fold {k} ______")
    combo_model.eval()
    
    total = 0
    correct = 0

    #process the input data
    idx = np.random.randint(0, Tests, size=batch_size)
    xvalid = ValidateX.view(-1, InputDim, SeqDim)
    xvalid = xvalid.to('cuda:0')
    yvalid = ValidateY

    #forward prop
    predict = combo_model(xvalid.float())
    
    #predict
    values, predicted_classes = torch.max(predict, 1)
    expected_classes = map_to_class(yvalid)        
    correct += (1*(predicted_classes == expected_classes.to('cuda:0'))).sum()
    total +=float(predicted_classes.size(0))
    
    #evaluate
    validation_accuracy = 100*(correct/total)
    training_accuracy = 100*(train_correct/train_total)

    print(f"correct: {correct}, total: {total}")
    #print(f"prediction: {predicted_classes}\nexpected_classes: {expected_classes}")
    print(f"Iteration: {iter_}, Loss: {probs.item()}, Validation Accuracy: {validation_accuracy}%, Training Accuracy: {training_accuracy}%\n")  

    #store info for graphing later
    validation_store.append(validation_accuracy)
    loss_store.append(probs)
    training_store.append(training_accuracy)

    #store the model
    model_store.append(copy.deepcopy(combo_model))

    #iterate the validate fold
    validate_fold +=1
    
    #--- Early stopping criterion here --- #
    if validation_accuracy > 73.0:
      #store
      print("Early Stopping!")
      stop_now = True
      break
    


...... Training for Epoch 0 ......
______ Training for k-folds [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14] ______
______ Validating for k-fold 0 ______
correct: 86, total: 141.0
Iteration: 1500, Loss: 0.5384140610694885, Validation Accuracy: 60.9929084777832%, Training Accuracy: 82.61333465576172%

______ Training for k-folds [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14] ______
______ Validating for k-fold 1 ______
correct: 75, total: 141.0
Iteration: 3000, Loss: 0.23323768377304077, Validation Accuracy: 53.191490173339844%, Training Accuracy: 82.02777862548828%

______ Training for k-folds [ 0  1  3  4  5  6  7  8  9 10 11 12 13 14] ______
______ Validating for k-fold 2 ______
correct: 79, total: 141.0
Iteration: 4500, Loss: 0.17481370270252228, Validation Accuracy: 56.02836608886719%, Training Accuracy: 82.05999755859375%

______ Training for k-folds [ 0  1  2  4  5  6  7  8  9 10 11 12 13 14] ______
______ Validating for k-fold 3 ______
correct: 89, total: 141.0
Iteration: 6000, Loss:

In [76]:
#Find Best Model and Test It
best_ind = np.argmax(np.array(validation_store))
best_model = model_store[best_ind]


print(f"______ Evaluating On Test Data ______")
best_model.eval()

total = 0
correct = 0

#process the input data
xtest = Xtest.view(-1, InputDim, SeqDim)
xtest = xtest.to('cuda:0')
ytest = Ytest

#forward prop
predict = best_model(xtest.float())

#predict
values, predicted_classes = torch.max(predict, 1)
expected_classes = map_to_class(ytest)        
correct += (1*(predicted_classes == expected_classes.to('cuda:0'))).sum()
total +=float(predicted_classes.size(0))

#evaluate
test_accuracy = 100*(correct/total)

print(f"correct: {correct}, total: {total}")
print(f"Testing Accuracy: {test_accuracy}%")  


______ Evaluating On Test Data ______
correct: 257, total: 443.0
Testing Accuracy: 58.013545989990234%
